In [67]:
import os
from lxml import etree as lxml
import zipfile, itertools
import io
from officelib.xllib import *

In [68]:
os.chdir("C:\\Users\\Nathan\\Downloads")

In [69]:
def load_xml(z):
    with z.open("archive_manifest.xml", 'r') as f:
        tree = lxml.parse(f)
    return tree

def load_zip(file):
    return zipfile.ZipFile(file)

In [71]:
tree = lxml.parse("test\\ViewOnlySecurity.xml")
root = tree.getroot()
for c in root.findall("security-group"):
    if c.find("name").text == "View":
        break
else:
    raise ValueError("oops")

In [72]:
xl = Excel()
wb = None
_wb = None
for _wb in xl.Workbooks:
    if _wb.Name == "helix_security.xlsx":
        wb = _wb
        break
del _wb
if wb is None:
    wb = xl.Workbooks.Add()
    wb.Name = "helix_security.xlsx"
ws = wb.Worksheets(1)
cells = ws.Cells
cr = cells.Range

def switch_sheet(id):
    global ws, cells, cr
    ws = wb.Worksheets(id)
    cells = ws.Cells
    cr = cells.Range

In [73]:
def paste(cell, data):
    c2 = cell.GetOffset(len(data) - 1, len(data[0]) - 1)
    target = cell.Range(cell, c2)
    target.NumberFormat = "@"
    target.Value = data
    

In [63]:
tree = lxml.parse("test\\security_groups_out.xml")
root = tree.getroot()

In [65]:
data = []
for cat in root.xpath("security-group/command-categories/command-category"):
    attrib = cat.attrib
    name = attrib["name"]
    data.append((name, ", ".join(attrib.keys())))

ws.UsedRange.Clear()
cr("A1:C1").Value = [("Command", "Key", "Value")]
paste(cr("A2"),data)

In [66]:
for cmd in root.xpath("security-group/command-categories/command-category[./command-subtype]"):
    print(cmd.attrib['name'])

Artifacts


In [20]:
newroot = lxml.Element(root.tag, root.attrib)
newroot.text = root.text
newroot.tail = root.tail
newroot.append(c)

In [24]:
def create_template(name, description):
    doc = lxml.Element("TestTrackData", version="1638912")
    
    sec = lxml.SubElement(doc, "security-group")
    lxml.SubElement(sec, "name").text = name
    lxml.SubElement(sec, "description").text = description
    lxml.SubElement(sec, "users-in-popup").text = "Yes"
    lxml.SubElement(sec, "customers-in-popup").text = "No"
    
    lxml.SubElement(sec, "command-categories")
    lxml.SubElement(sec, "field-security")
    return doc

def add_commands_to_category(category, commands):
    for name, enabled in commands:
        if isinstance(enabled, bool):
            enabled = str(enabled).lower()
        lxml.SubElement(category, "command", enabled=enabled).text = name

def add_basic_command_category(cats_elem, name: str, events: str, commands: list):
    cat = lxml.SubElement(cats_elem, name=name, events=events)
    add_commands_to_category(cat, commands)
    
def add_type_category(cats_elem, name, commands, onlyReportingUser="no", workWithLabel="", filter="$lt;All&gt;"):
    attrib = {
        'name': name,
        'nameLowerCase': name.lower(),
        'events': "false",
        'onlyReportingUser': onlyReportingUser,
        'workWithLabel': workWithLabel,
        'filter': filter
    }
    cat = lxml.SubElement(cats_elem, "command-category", attrib)
    add_commands(cat, commands)
        
def create_artifacts_category(cats_elem, commands, onlyReportingUser="no", workWithLabel="All Artifacts", filter="$lt;All&gt;"):
    attrib = {
        'name': "Artifacts",
        'nameLowerCase': "artifacts",
        'events': "false",
        'onlyReportingUser': onlyReportingUser,
        'workWithLabel': workWithLabel,
        'filter': filter
    }
    return lxml.SubElement(cats_elem, "command-category", attrib)

def add_artifacts_subtype(artifacts, name, override_default="false", subtype_id="0"):
    
    
    

TypeError: 'type' object is not subscriptable

In [21]:
newtree = lxml.ElementTree(newroot)
with open("test\\security_groups_out.xml", 'wb') as f:
    newtree.write(f, pretty_print=True, xml_declaration=True, standalone=False, encoding='UTF-8', 
                  doctype=b'<!DOCTYPE TestTrackData SYSTEM "TestTrackData.dtd">')